In [1]:
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
import os
from sqlalchemy import create_engine
import json

In [2]:
# Cria o diretório
os.makedirs('./enem2019', exist_ok=True)

In [3]:
"""
# Define a url
url = 'http://download.inep.gov.br/microdados/microdados_enem_2019.zip'

# Download do conteudo
filebytes = BytesIO(
    requests.get(url).content
)

# Extrarir o conteudo do zipfile
myzip = zipfile.ZipFile(filebytes)
myzip.extractall('./enem2019')
"""

"\n# Define a url\nurl = 'http://download.inep.gov.br/microdados/microdados_enem_2019.zip'\n\n# Download do conteudo\nfilebytes = BytesIO(\n    requests.get(url).content\n)\n\n# Extrarir o conteudo do zipfile\nmyzip = zipfile.ZipFile(filebytes)\nmyzip.extractall('./enem2019')\n"

In [4]:
csv_file = './enem2019/DADOS/MICRODADOS_ENEM_2019.csv'

In [5]:
number_lines = sum(1 for row in (open(csv_file, encoding='ISO-8859-1')))

In [6]:
# FROM https://gist.github.com/jrivero/1085501

def split(filehandler, delimiter=',', row_limit=1000,
          output_name_template='output_%s.csv', output_path='.', keep_headers=True):
    import csv
    reader = csv.reader(filehandler, delimiter=delimiter)
    current_piece = 1
    current_out_path = os.path.join(
        output_path,
        output_name_template % current_piece
    )
    current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
    current_limit = row_limit
    if keep_headers:
        headers = next(reader)
        current_out_writer.writerow(headers)
    for i, row in enumerate(reader):
        if i + 1 > current_limit:
            current_piece += 1
            current_limit = row_limit * current_piece
            current_out_path = os.path.join(
                output_path,
                output_name_template % current_piece
            )
            current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
            if keep_headers:
                current_out_writer.writerow(headers)
        current_out_writer.writerow(row)

In [34]:
split(open(csv_file, encoding='ISO-8859-1'), delimiter=';', row_limit=20000,
     output_name_template='microdados_enem_%s.csv', output_path='./enem2019/DADOS/splitted_files', keep_headers=True)

In [7]:
# Get access variables
with open('vaccess_pg.txt') as access_file:
    vaccess = json.load(access_file)

In [8]:
# Create connection
conn_str = 'postgresql+psycopg2://'+vaccess['user']+':'+vaccess['password']+'@'+vaccess['host:port']+'/'+vaccess['dbname']
engine_pg = create_engine(conn_str)

In [ ]:
directory = './enem2019/DADOS/splitted_files/'

for filename in sorted(os.listdir(directory)):
    print(filename)
    if filename.endswith(".csv"):
        enem = pd.read_csv(
                directory+filename, sep = ';', decimal = '.', encoding='ISO-8859-1'
        )
        try:
            enem.to_sql(
                'enem',
                engine_pg,
                index=False,
                chunksize=10000,
                if_exists='append'
            )
        finally:
            # Close connection
            engine_pg.dispose()
        continue
    else:
        continue

In [ ]:
import psycopg2
conn = psycopg2.connect("host=172.18.0.2 dbname=postgres user=postgres password=postgres")
cur = conn.cursor()


directory = './enem2019/DADOS/splitted_files/'

for filename in sorted(os.listdir(directory)):
    print(filename)
    if filename.endswith(".csv"):
        enem = pd.read_csv(
                directory+filename, sep = ';', decimal = '.', encoding='ISO-8859-1'
        )
        try:
            with open(directory+filename, 'r') as f:
                # Notice that we don't need the `csv` module.
                next(f) # Skip the header row.
                cur.copy_from(f, 'enem', sep=';', null="")

            conn.commit()
        finally:
            # Close connection
            engine_pg.dispose()
        continue
    else:
        continue